# Auto-Batched Joint Distributions: A Gentle Tutorial

##### Copyright 2020 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/probability/examples/JointDistributionAutoBatched_A_Gentle_Tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

### Introdução

O TensorFlow Probability (TFP) conta com diversas abstrações de `JointDistribution` que facilitam a inferência probabilística ao permitir que o usuário expresse com facilidade um modelo gráfico probabilístico de uma forma quase matemática. A abstração gera métodos para fazer a amostragem do modelo e avaliar a probabilidade logarítmica das amostras. Neste tutorial, vamos avaliar as variantes com "divisão automática em lotes", que foram desenvolvidas depois das abstrações de `JointDistribution` originais. Com relação às originais, as abstrações sem divisão automática em lotes, as versões com divisão automática em lotes são mais simples de usar e mais ergonômicas, permitindo que diversos modelos sejam expressados com menos código boilerplate. Neste Colab, veremos um modelo simples com muitos detalhes (talvez tediosos), esclarecendo os problemas que a divisão automática em lotes resolve e ensinando mais sobre os conceitos de formato do TFP para os leitores.

Antes do lançamento da divisão automática em lotes, havia algumas variantes diferentes de `JointDistribution`, correspondentes a estilos sintáticos diferentes para expressar modelos probabilísticos: `JointDistributionSequential`, `JointDistributionNamed` e `JointDistributionCoroutine`. A divisão automática em lotes existe como mixin, então agora temos variantes `AutoBatched` de todas essas distribuições. Neste tutorial, veremos as diferenças entre `JointDistributionSequential` e `JointDistributionSequentialAutoBatched`. Porém, tudo o que fizermos aqui pode ser aplicado às outras variantes basicamente sem alterações.


### Dependências e pré-requisitos


In [ ]:
#@title Import and set ups{ display-mode: "form" }

import functools
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_probability as tfp

tfd = tfp.distributions

### Pré-requisito: problema de regressão bayesiana

Vamos considerar um cenário bem simples de regressão bayesiana:

$$
\begin{align*}
m & \sim \text{Normal}(0, 1) \\
b & \sim \text{Normal}(0, 1) \\
Y & \sim \text{Normal}(mX + b, 1)
\end{align*}
$$

Nesse modelo, `m` e `b` são obtidos a partir das distribuições normais padrão, e as observações `Y` são obtidas a partir de uma distribuição normal cuja média depende das variáveis aleatórias `m` e `b` e de algumas covariáveis (conhecidas, não aleatórias) `X` (por questões de simplicidade, neste exemplo, pressupomos que a escala de todas as variáveis aleatórias seja conhecida).

Para fazer a inferência nesse modelo, precisaríamos saber tanto as covariáveis `X` quanto as observações `Y`, mas, para a finalidade deste tutorial, precisaremos somente de `X` e, portanto, definimos um `X` simples:

In [ ]:
X = np.arange(7)
X

array([0, 1, 2, 3, 4, 5, 6])

### Dados desejados

Na inferência probabilística, geralmente queremos realizar duas operações básicas:

- `sample`: obtenção de amostras do modelo.
- `log_prob`: computação da probabilidade logarítmica de uma amostra do modelo.

A principal contribuição das abstrações de `JointDistribution` do TFP (bem como de diversas outras estratégias de programação probabilística) é permitir que os usuários escrevam um modelo *uma vez* e tenham acesso tanto a computações de `sample` quanto de `log_prob`.

Observando que temos sete pontos em nosso dataset (`X.shape = (7,)`), agora podemos declarar os dados desejados para uma `JointDistribution` excelente:

- `sample()` deve gerar uma lista de `Tensors` com formato `[(), (), (7,)]`, correspondente ao declive escalar, ao bias escalar e às observações do vetor, respectivamente.
- `log_prob(sample())` deve gerar um escalar: a probabilidade logarítmica de um declive, bias e observações específicos.
- `sample([5, 3])` deve gerar uma lista de `Tensors` com formato `[(5, 3), (5, 3), (5, 3, 7)]`, representando um *lote* `(5, 3)` de amostras do modelo.
- `log_prob(sample([5, 3]))` deve gerar um `Tensor` com formato (5, 3).

Agora vamos conferir uma sequência de modelos de `JointDistribution`, ver como alcançar os dados desejados acima e quem sabe aprender um pouco mais sobre os formatos do TFP.

Spoiler: a estratégia que satisfaz os dados desejados acima sem código boilerplate adicional é a [divisão automática em lotes](#scrollTo=_h7sJ2bkfOS7). 

### Primeira tentativa: `JointDistributionSequential`

In [ ]:
jds = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

Isso é basicamente uma tradução direta do modelo em código. O declive `m` e o bias `b` são diretos. `Y` é definido usando-se uma função `lambda`: o padrão geral é que uma função `lambda` de $k$ argumentos em uma distribuição `JointDistributionSequential` (JDS, na sigla em inglês) use as $k$ distribuições anteriores do modelo. Agora, a ordem "reversa".

Vamos chamar `sample_distributions`, que retorna tanto uma amostra *quanto* as "subdistribuições" subjacentes que foram usadas para gerar a amostra (poderíamos ter gerado somente a amostra chamando `sample`; posteriormente neste tutorial, também será conveniente ter as distribuições). A amostra que geramos é boa:

In [ ]:
dists, sample = jds.sample_distributions()
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

Mas `log_prob` gera um resultado com formato indesejado:

In [ ]:
jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([-4.4777603, -4.6775575, -4.7430477, -4.647725 , -4.5746684,
       -4.4368567, -4.480562 ], dtype=float32)>

E a amostragem múltipla não funciona:

In [ ]:
try:
  jds.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


Vamos tentar entender o que está dando errado.

### Breve revisão: formato de lote e evento

No TFP, uma distribuição de probabilidades comum (não uma `JointDistribution`) tem um *formato de evento* e um *formato de lote*, e compreender a diferença entre eles é essencial para usar o TFP com eficácia:

- Cada formato descreve o formato de uma única obtenção de dado da distribuição; a obtenção pode ser dependente entre as dimensões. Para distribuições escalares, o formato de evento é []. Para uma distribuição normal multivariada de 5 dimensões, o formato de evento é [5].
- O formato de lote descreve obtenções de dados independentes, não distribuídas identicamente, ou seja, um "lote" de distribuições. Representar um lote de distribuições em um único objeto do Python é uma das principais formas de o TFP alcançar eficiência em grande escala.

Para nossos propósitos, um fato crítico que deve ser lembrando é que, se chamarmos `log_prob` em uma única amostra de uma distribuição, o resultado sempre terá um formato que coincide (ou seja, tem as dimensões da extremidade direita) com o formato de *lote*.

Confira mais detalhes sobre formatos no [tutorial "Noções básicas sobre os formatos de distribuições do TensorFlow"](https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes).


### Por que `log_prob(sample())` não gera um escalar? 

Vamos usar nossos conhecimentos sofre formato de lote e evento para ver o que está acontecendo com `log_prob(sample())`. Veja a amostra novamente:

In [ ]:
sample

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.668757>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6585061>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([ 0.18573815, -1.79962   , -1.8106272 , -3.5971394 , -6.6625295 ,
        -7.308844  , -9.832693  ], dtype=float32)>]

E aqui estão as distribuições:

In [ ]:
dists

[<tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>]

A probabilidade logarítmica é computada somando os probabilidades logarítmicas das subdistribuições nos elementos (coincidentes) das partes:

In [ ]:
log_prob_parts = [dist.log_prob(s) for (dist, s) in zip(dists, sample)]
log_prob_parts

[<tf.Tensor: shape=(), dtype=float32, numpy=-2.3113134>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-1.1357536>,
 <tf.Tensor: shape=(7,), dtype=float32, numpy=
 array([-1.0306933, -1.2304904, -1.2959809, -1.200658 , -1.1276014,
        -0.9897899, -1.0334952], dtype=float32)>]

In [ ]:
sum(log_prob_parts) - jds.log_prob(sample)

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

Portanto, a primeira explicação é que o cálculo da probabilidade logarítmica está retornando um Tensor de dimensão 7 porque o terceiro subcomponente de `log_prob_parts` é um Tensor dimensão 7. Mas por quê?

Podemos ver que o último elemento de `dists`, que corresponde à nossa distribuição para `Y` na formulação matemática, tem um `batch_shape` igual a `[7]`. Em outras palavras, a distribuição para `Y` é um lote de 7 distribuições normais independentes (com médias diferentes e, neste caso, a mesma escala).

Agora entendemos o que está errado: na JDS, a distribuição para `Y` tem `batch_shape=[7]`, uma amostra da JDS representa escalares para `m` e `b` e um "lote" de 7 distribuições normais independentes. Além disso, `log_prob` computa 7 probabilidades logarítmicas separadas, cada uma representando a probabilidade logarítmica de obter `m` e `b` e uma única observação `Y[i]` em algum `X[i]`.

### Corrigindo `log_prob(sample())` com `Independent`

Lembre-se de que `dists[2]` tem `event_shape=[]` e `batch_shape=[7]`:

In [ ]:
dists[2]

<tfp.distributions.Normal 'JointDistributionSequential_sample_distributions_Normal' batch_shape=[7] event_shape=[] dtype=float32>

Usando a metadistribuição `Independent` do TFP, que converte dimensões de lote em dimensões de evento, podemos convertê-la em uma distribuição com `event_shape=[7]` e `batch_shape=[]` (vamos renomear para `y_dist_i`, pois é uma distribuição para `Y`, com `_i` indicando nosso encapsulamento `Independent`): 

In [ ]:
y_dist_i = tfd.Independent(dists[2], reinterpreted_batch_ndims=1)
y_dist_i

<tfp.distributions.Independent 'IndependentJointDistributionSequential_sample_distributions_Normal' batch_shape=[] event_shape=[7] dtype=float32>

Agora, a `log_prob` de um vetor de dimensão 7 é um escalar:

In [ ]:
y_dist_i.log_prob(sample[2])

<tf.Tensor: shape=(), dtype=float32, numpy=-7.9087086>

Nos bastidores, `Independent` soma ao longo do lote:

In [ ]:
y_dist_i.log_prob(sample[2]) - tf.reduce_sum(dists[2].log_prob(sample[2]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

E podemos usá-la para construir uma nova `jds_i` (novamente, `i` significa `Independent`), em que `log_prob` retorna um escalar:

In [ ]:
jds_i = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m*X + b, scale=1.),
        reinterpreted_batch_ndims=1)
])

jds_i.log_prob(sample)

<tf.Tensor: shape=(), dtype=float32, numpy=-11.355776>

Algumas observações:

- `jds_i.log_prob(s)` *não* é a mesma que `tf.reduce_sum(jds.log_prob(s))`. A primeira gera a probabilidade logarítmica "correta" da distribuição conjunta. A segunda faz a soma para um Tensor de dimensão 7, em que cada elemento é a soma da probabilidade logarítmica de `m`, `b` e um único elemento da probabilidade logarítmica de `Y`, então `m` e `b` são contabilizados acima do que deveriam ser. (`log_prob(m) + log_prob(b) + log_prob(Y)` retorna um resultado em vez de gerar uma exceção porque o TFP segue as regras de broadcast do TF e do NumPy – adicionar um escalar a um vetor gera um resultado com tamanho do vetor).
- Neste caso específico, poderíamos ter resolvido o problema e conseguido o mesmo resultado usando `MultivariateNormalDiag` em vez de `Independent(Normal(...))`. `MultivariateNormalDiag` é uma distribuição com valor de vetor (ou seja, já tem formato de evento como vetor). De fato, `MultivariateNormalDiag` poderia ser (mas não é) implementada como uma composição de `Independent` e `Normal`. É importante lembrar que, dado um vetor `V`, as amostras de `n1 = Normal(loc=V)` e `n2 = MultivariateNormalDiag(loc=V)` são indistinguíveis. A diferença entre essas distribuições é que `n1.log_prob(n1.sample())` é um vetor, e `n2.log_prob(n2.sample())` é um escalar.

### Múltiplas amostras?

Obter múltiplas amostras não funciona também:

In [ ]:
try:
  jds_i.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


Vamos pensar no motivo. Quando chamamos `jds_i.sample([5, 3])`, primeiro obtemos amostras para `m` e `b`, cada uma com formato `(5, 3)`. Em seguida, tentamos construir uma distribuição `Normal` via:

```
tfd.Normal(loc=m*X + b, scale=1.)
```

Mas, se `m` tem formato `(5, 3)`, e `X` tem formato `7`, não podemos multiplicá-los e, de fato, é esse erro que estamos obtendo:

In [ ]:
m = tfd.Normal(0., 1.).sample([5, 3])
try:
  m * X
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3] vs. [7] [Op:Mul]


Para resolver esse problema, vamos pensar em que propriedades a distribuição para `Y` precisa ter. Se chamarmos `jds_i.sample([5, 3])`, então sabemos que tanto `m` quanto `b` terão formato igual a `(5, 3)`. Que formato uma chamada a `sample` na distribuição `Y` gera? A resposta óbvia é `(5, 3, 7)`: para cada ponto do lote, queremos uma amostra com o mesmo tamanho que o de `X`. Podemos conseguir isso usando as funcionalidades de broadcast do TensorFlow, acrescentando dimensões extras:

In [ ]:
m[..., tf.newaxis].shape

TensorShape([5, 3, 1])

In [ ]:
(m[..., tf.newaxis] * X).shape

TensorShape([5, 3, 7])

Acrescentando um eixo tanto a `m` quanto a `b`, podemos definir uma nova JDS com suporte a múltiplas amostras:

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

shaped_sample = jds_ia.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-1.1133379 ,  0.16390413, -0.24177533],
        [-1.1312429 , -0.6224666 , -1.8182136 ],
        [-0.31343174, -0.32932565,  0.5164407 ],
        [-0.0119963 , -0.9079621 ,  2.3655841 ],
        [-0.26293617,  0.8229698 ,  0.31098196]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[-0.02876974,  1.0872147 ,  1.0138507 ],
        [ 0.27367726, -1.331534  , -0.09084719],
        [ 1.3349475 , -0.68765205,  1.680652  ],
        [ 0.75436825,  1.3050154 , -0.9415123 ],
        [-1.2502679 , -0.25730947,  0.74611956]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 7), dtype=float32, numpy=
 array([[[-1.8258233e+00, -3.0641669e-01, -2.7595463e+00, -1.6952467e+00,
          -4.8197951e+00, -5.2986512e+00, -6.6931367e+00],
         [ 3.6438566e-01,  1.0067395e+00,  1.4542470e+00,  8.1155670e-01,
           1.8868095e+00,  2.3877139e+00,  1.0195159e+00],
         [-8.3624744e-01,  1.2518480e+00,  1.0943471e+00, 

In [ ]:
jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.483114 , -10.139662 , -11.514159 ],
       [-11.656767 , -17.201958 , -12.132455 ],
       [-17.838818 ,  -9.474525 , -11.24898  ],
       [-13.95219  , -12.490049 , -17.123957 ],
       [-14.487818 , -11.3755455, -10.576363 ]], dtype=float32)>

Como checagem extra, vamos verificar se a probabilidade logarítmica para um único ponto do lote coincide com a que obtivemos antes:

In [ ]:
(jds_ia.log_prob(shaped_sample)[3, 1] -
 jds_i.log_prob([shaped_sample[0][3, 1],
                 shaped_sample[1][3, 1],
                 shaped_sample[2][3, 1, :]]))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

<a id="AutoBatching-For-The-Win"></a>

### Divisão automática em lotes é a melhor opção


Excelente! Agora temos uma versão da JointDistribution que trata todo os nossos dados desejados: `log_prob` retorna um escalar graças ao uso de `tfd.Independent`, e agora múltiplas amostras funcionam porque fixamos o broadcast ao adicionar eixos extras.

E se dissermos que existe uma forma melhor e mais fácil? Ela é chamada `JointDistributionSequentialAutoBatched` (JDSAB, na sigla em inglês):

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

In [ ]:
jds_ab.log_prob(jds.sample())

<tf.Tensor: shape=(), dtype=float32, numpy=-12.954952>

In [ ]:
shaped_sample = jds_ab.sample([5, 3])
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-12.191533 , -10.43885  , -16.371655 ],
       [-13.292994 , -11.97949  , -16.788685 ],
       [-15.987699 , -13.435732 , -10.6029   ],
       [-10.184758 , -11.969714 , -14.275676 ],
       [-12.740775 , -11.5654125, -12.990162 ]], dtype=float32)>

In [ ]:
jds_ab.log_prob(shaped_sample) - jds_ia.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

Como funciona? Você pode [avaliar o código](https://github.com/tensorflow/probability/blob/main/tensorflow_probability/python/distributions/joint_distribution_auto_batched.py#L426) para entender melhor, mas vamos apresentar uma visão geral breve que é suficiente para a maioria dos casos de uso:

- Lembre-se de que o primeiro problema era que a distribuição para `Y` tinha `batch_shape=[7]` e `event_shape=[]`, e usamos `Independent` para converter a dimensão de lote em dimensão de evento. A JDSAB ignora os formatos de lote das distribuições componentes; em vez disso, ela trata o formato de lote como uma propriedade geral do modelo, que é presumido como `[]` (a menos que seja especificado de outra forma definindo `batch_ndims > 0`). O efeito é equivalente a usar tfd.Independent para converter *todas* as dimensões de lote das distribuições componentes em dimensões de evento, conforme fizemos manualmente acima.
- O segundo problema era a necessidade de alterar os formatos de `m` e `b` para que pudessem fazer o broadcast corretamente com `X` ao criar múltiplas amostras. Com a JDSAB, escrevemos um modelo para gerar uma única amostra e fazemos o modelo inteiro gerar múltiplas amostras usando [vectorized_map](https://www.tensorflow.org/api_docs/python/tf/vectorized_map) do TensorFlow (essa funcionalidade é análoga a [vmap](https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#Auto-vectorization-with-vmap) do JAX).

Explorando o problema do formato de lote com mais detalhes, podemos comparar os formatos de lote da distribuição conjunta original "ruim" `jds`, das distribuições com lote fixo `jds_i` e `jds_ia`, e da `jds_ab` com divisão automática em lotes:

In [ ]:
jds.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([7])]

In [ ]:
jds_i.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ia.batch_shape

[TensorShape([]), TensorShape([]), TensorShape([])]

In [ ]:
jds_ab.batch_shape

TensorShape([])

Podemos ver que a `jds` original tem subdistribuições com formatos de lote diferentes. `jds_i` e `jds_ia` corrigem esse problema criando subdistribuições com o mesmo formato de lote (vazio). `jds_ab` tem somente um único formato de lote (vazio).

É importante salientar que `JointDistributionSequentialAutoBatched` oferece uma generalização adicional gratuitamente. Vamos supor que tornemos as covariáveis `X` (e, implicitamente, as observações `Y`) bidimensionais:

In [ ]:
X = np.arange(14).reshape((2, 7))
X

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13]])

A `JointDistributionSequentialAutoBatched` funciona sem alterações (precisamos redefinir o modelo porque é feito cache do formato de `X` por `jds_ab.log_prob`):

In [ ]:
jds_ab = tfd.JointDistributionSequentialAutoBatched([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Normal(loc=m*X + b, scale=1.) # Y
])

shaped_sample = jds_ab.sample([5, 3])
shaped_sample

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.1813647 , -0.85994506,  0.27593774],
        [-0.73323774,  1.1153806 ,  0.8841938 ],
        [ 0.5127983 , -0.29271227,  0.63733214],
        [ 0.2362284 , -0.919168  ,  1.6648189 ],
        [ 0.26317367,  0.73077047,  2.5395133 ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 0.09636458,  2.0138032 , -0.5054413 ],
        [ 0.63941646, -1.0785882 , -0.6442188 ],
        [ 1.2310615 , -0.3293852 ,  0.77637213],
        [ 1.2115169 , -0.98906034, -0.07816773],
        [-1.1318136 ,  0.510014  ,  1.036522  ]], dtype=float32)>,
 <tf.Tensor: shape=(5, 3, 2, 7), dtype=float32, numpy=
 array([[[[-1.9685398e+00, -1.6832136e+00, -6.9127172e-01,
            8.5992378e-01, -5.3123581e-01,  3.1584005e+00,
            2.9044402e+00],
          [-2.5645006e-01,  3.1554163e-01,  3.1186538e+00,
            1.4272424e+00,  1.2843871e+00,  1.2266440e+00,
            1.2798605e+00]],
 
         [[ 1.5973477e+00,

In [ ]:
jds_ab.log_prob(shaped_sample)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[-28.90071 , -23.052422, -19.851362],
       [-19.775568, -25.894997, -20.302256],
       [-21.10754 , -23.667885, -20.973007],
       [-19.249458, -20.87892 , -20.573763],
       [-22.351208, -25.457762, -24.648403]], dtype=float32)>

Por outro lado, a `JointDistributionSequential` criada cuidadosamente deixa de funcionar:

In [ ]:
jds_ia = tfd.JointDistributionSequential([
    tfd.Normal(loc=0., scale=1.),   # m
    tfd.Normal(loc=0., scale=1.),   # b
    lambda b, m: tfd.Independent(   # Y
        tfd.Normal(loc=m[..., tf.newaxis]*X + b[..., tf.newaxis], scale=1.),
        reinterpreted_batch_ndims=1)
])

try:
  jds_ia.sample([5, 3])
except tf.errors.InvalidArgumentError as e:
  print(e)

Incompatible shapes: [5,3,1] vs. [2,7] [Op:Mul]


Para corrigir esse problema, precisamos adicionar um segundo `tf.newaxis` tanto a `m` quanto a `b` para deixar o formato igual, e aumentar `reinterpreted_batch_ndims` para 2 na chamada a `Independent`. Neste caso, deixar o mecanismo de divisão automática em lotes tratar os problemas de formato é mais rápido, fácil e ergonômico.

Novamente, observe que, embora este notebook tenha explorado `JointDistributionSequentialAutoBatched`, as outras variantes de `JointDistribution` têm `AutoBatched` equivalente (para usuários de `JointDistributionCoroutine`, `JointDistributionCoroutineAutoBatched` tem o benefício adicional de não precisar mais especificar nós `Root`; se você nunca tiver usado `JointDistributionCoroutine`, pode ignorar esta afirmação sem problema nenhum).

### Consideração final

Neste notebook, apresentamos `JointDistributionSequentialAutoBatched` e detalhamos um exemplo simples. Esperamos que você tenha aprendido alguma coisa sobre os formatos do TFP e a divisão automática em lotes!